In [56]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LogisticRegression, Lasso, LinearRegression,  ElasticNet
from sklearn.model_selection import train_test_split,  cross_val_score, KFold,  RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, StratifiedGroupKFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree, DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


In [55]:
arquivo = pd.read_csv('/home/marco/projects/machine_learning/ML/exFinal/archive/recipeData.csv', encoding="iso-8859-1")
arquivo.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


In [41]:
faltantes_percentual = (arquivo.isnull().sum()/len(arquivo['StyleID']))*100
print(faltantes_percentual)

BeerID            0.000000
Name              0.002708
URL               0.000000
Style             0.806921
StyleID           0.000000
Size(L)           0.000000
OG                0.000000
FG                0.000000
ABV               0.000000
IBU               0.000000
Color             0.000000
BoilSize          0.000000
BoilTime          0.000000
BoilGravity       4.048144
Efficiency        0.000000
MashThickness    40.432705
SugarScale        0.000000
BrewMethod        0.000000
PitchRate        53.143066
PrimaryTemp      30.681957
PrimingMethod    90.847673
PrimingAmount    93.536508
UserId           68.358132
dtype: float64


In [69]:

# Remover colunas com mais de 60% de valores NaN
if len(arquivo) > 0:
    columns_nan = [col for col in arquivo.columns if (arquivo[col].isnull().sum() / len(arquivo)) * 100 > 60]
    arquivo = arquivo.drop(columns=columns_nan)

# Calcular percentual de valores faltantes
faltantes_percentual = (arquivo.isnull().sum() / len(arquivo)) * 100 if len(arquivo) > 0 else None
print(faltantes_percentual)

# Remover colunas do tipo 'object'
object_columns = [col for col in arquivo.select_dtypes(include=['object']).columns]
arquivo = arquivo.drop(columns=object_columns)

# Preencher colunas numéricas com a média (exceto PitchRate)
if len(arquivo) > 0:
    columns_switch = [col for col in arquivo.columns if arquivo[col].isnull().sum() > 0]
    for k in columns_switch:
        if k != 'PitchRate':  # Evita modificar 'PitchRate'
            arquivo[k] = arquivo[k].fillna(arquivo[k].mean())
arquivo['MashThickness'] = arquivo['MashThickness'].fillna(arquivo['MashThickness'].mean())

BeerID            0.000000
StyleID           0.000000
Size(L)           0.000000
OG                0.000000
FG                0.000000
ABV               0.000000
IBU               0.000000
Color             0.000000
BoilSize          0.000000
BoilTime          0.000000
BoilGravity       0.000000
Efficiency        0.000000
MashThickness     0.000000
PitchRate        53.143066
PrimaryTemp       0.000000
dtype: float64


In [70]:
faltantes_percentual = (arquivo.isnull().sum()/len(arquivo['StyleID']))*100
print(faltantes_percentual)

BeerID            0.000000
StyleID           0.000000
Size(L)           0.000000
OG                0.000000
FG                0.000000
ABV               0.000000
IBU               0.000000
Color             0.000000
BoilSize          0.000000
BoilTime          0.000000
BoilGravity       0.000000
Efficiency        0.000000
MashThickness     0.000000
PitchRate        53.143066
PrimaryTemp       0.000000
dtype: float64


In [65]:
arquivo.dtypes


BeerID             int64
Name              object
URL               object
Style             object
StyleID            int64
Size(L)          float64
OG               float64
FG               float64
ABV              float64
IBU              float64
Color            float64
BoilSize         float64
BoilTime           int64
BoilGravity      float64
Efficiency       float64
MashThickness    float64
SugarScale        object
BrewMethod        object
PitchRate        float64
PrimaryTemp      float64
PrimingMethod     object
PrimingAmount     object
UserId           float64
dtype: object

In [71]:
# Criando conjuntos de treino e preenchimento sem duplicar filtros
x_treino = arquivo[arquivo['PitchRate'].notnull()].drop('PitchRate', axis=1)
y_treino = arquivo.loc[arquivo['PitchRate'].notnull(), 'PitchRate']

x_preench = arquivo[arquivo['PitchRate'].isnull()].drop('PitchRate', axis=1)

y_preench = arquivo.loc[arquivo['PitchRate'].isnull(), 'PitchRate']


In [74]:
modelo = DecisionTreeRegressor()
modelo.fit(x_treino, y_treino)
y_preench = modelo.predict(x_preench)

In [75]:
arquivo.PitchRate[arquivo.PitchRate.isnull()] = y_preench

/tmp/ipykernel_275029/3541562083.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  arquivo.PitchRate[arquivo.PitchRate.isnull()] = y_preench
/tmp/ipykernel_275029/3541562083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a c

In [78]:
faltantes =  arquivo.isnull().sum()
faltantes_percentual = (arquivo.isnull().sum()/len(arquivo['StyleID']))*100
print(faltantes_percentual)

BeerID           0.0
StyleID          0.0
Size(L)          0.0
OG               0.0
FG               0.0
ABV              0.0
IBU              0.0
Color            0.0
BoilSize         0.0
BoilTime         0.0
BoilGravity      0.0
Efficiency       0.0
MashThickness    0.0
PitchRate        0.0
PrimaryTemp      0.0
dtype: float64


In [80]:
arquivo.head(15)


,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,PitchRate,PrimaryTemp
0,1,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038000,70.0,2.127235,0.75,17.780000
1,2,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070000,70.0,2.127235,0.75,19.175641
2,3,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.353955,70.0,2.127235,0.35,19.175641
3,4,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.353955,70.0,2.127235,0.35,19.175641
4,5,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050000,72.0,2.127235,0.75,19.000000
5,6,10,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047000,79.0,2.127235,1.00,19.175641
6,7,86,22.71,1.072,1.018,7.09,268.71,6.33,30.28,90,1.353955,75.0,2.127235,0.35,19.175641
7,8,45,20.82,1.054,1.014,5.36,19.97,5.94,28.39,75,1.040000,70.0,1.400000,0.50,19.175641
8,9,129,22.71,1.060,1.016,5.77,31.63,34.76,30.28,75,1.042000,73.0,2.127235,0.75,19.175641
9,10,86,20.82,1.080,1.017,8.22,93.02,8.29,28.39,60,1.058000,70.0,2.127235,0.50,21.110000
